In [13]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
pinecone_api_key = os.getenv("PINECONE_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

In [15]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
from pinecone import Pinecone, ServerlessSpec

indexName = "hybrid-search-langchain-pinecone"
pineconeClient = Pinecone(api_key = pinecone_api_key)

if indexName not in pineconeClient.list_indexes().names():
    pineconeClient.create_index(
        name = indexName,
        dimension = 384, 
        metric = 'dotproduct',
        spec = ServerlessSpec(cloud = 'aws', region = 'us-east-1'),
    )

In [16]:
index = pineconeClient.Index(indexName)
index

In [17]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name = 'all-MiniLM-l6-v2')
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-l6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [18]:
from pinecone_text.sparse import BM25Encoder

bm25Encoder = BM25Encoder().default()
bm25Encoder

In [19]:
sentences = [
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited Berlin"
]

bm25Encoder.fit(sentences)

100%|██████████| 3/3 [00:00<00:00, 213.52it/s]


In [20]:
bm25Encoder.dump('bm25_values.json')
bm25Encoder = BM25Encoder().load('bm25_values.json')